In [1]:
import pandas as pd
import requests
# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_colwidth", None)
from datetime import datetime, timedelta
import random
import yfinance as yf
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
import torch
model_id = "deepseek-ai/deepseek-llm-7b-chat"
torch.cuda.set_device(3)  # Sets default to GPU 0
device=torch.device("cuda:3")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map={"": 3},             # auto-distributes across GPUs
    torch_dtype="auto",            # picks bf16 or fp16 depending on availability
)

/home/f20222001/test-venv/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.25it/s]


In [ ]:
class fundamental_analyst:

    def __init__ (self, ticker):
        self.ticker=ticker
    
    def get_financial_info(self):
        ticker = yf.Ticker(self.ticker)
        self.income_stmt = ticker.financials
        self.balance_sheet = ticker.balance_sheet
        self.cash_flow = ticker.cashflow
        self.info = ticker.info
        self.year=self.income_stmt.columns[0]
        self.EPS=self.income_stmt[self.year]["Diluted EPS"]
        self.Net_Income=self.income_stmt[self.year]["Net Income"]
        self.Gross_Profit=self.income_stmt[self.year]["Gross Profit"]
        self.Revenue=self.income_stmt[self.year]["Total Revenue"]
        self.Total_Assets=self.balance_sheet[self.year]["Total Assets"]
        self.Total_Liabilities=self.balance_sheet[self.year]["Total Liabilities Net Minority Interest"]
        self.Shareholders_Equity=self.balance_sheet[self.year]["Stockholders Equity"]
        self.Free_Cash_Flow=self.cash_flow[self.year]["Free Cash Flow"]
        self.Investing_Cash_Flow=self.cash_flow[self.year]["Investing Cash Flow"]
        self.Financing_Cash_Flow=self.cash_flow[self.year]["Financing Cash Flow"]
        self.Operating_Cash_Flow=self.cash_flow[self.year]["Operating Cash Flow"]
        self.P_E=ticker.info['trailingPE']
        self.ROA=ticker.info['returnOnAssets']
        self.ROE=ticker.info['returnOnEquity']
    
    def generate_prompt(self):
        prompt = f"""
You are a fundamental investment analyst. Analyze the financial performance of {self.ticker} and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
Justify your decision in 4–6 bullet points using financial reasoning. Consider all the financial information shared. Only use the numerical data given. 
Do not add assumptions about company operations, reputation, or strategy.

Financials for {self.ticker}:

Income Statement:
Revenue: ${self.Revenue:,.0f}
Gross Profit: ${self.Gross_Profit:,.0f}
Net Income: ${self.Net_Income:,.0f}
EPS (Diluted): {self.EPS:.2f}

Balance Sheet:
Total Assets: ${self.Total_Assets:,.0f}
Total Liabilities: ${self.Total_Liabilities:,.0f}
Shareholders' Equity: ${self.Shareholders_Equity:,.0f}

Cash Flow:
Operating Cash Flow: ${self.Operating_Cash_Flow:,.0f}
Free Cash Flow: ${self.Free_Cash_Flow:,.0f}
Investing Cash Flow: ${self.Investing_Cash_Flow:,.0f}
Financing Cash Flow: ${self.Financing_Cash_Flow:,.0f}

Valuation and Ratios:
P/E Ratio: {self.P_E:.2f}
ROA: {self.ROA:.2%}
ROE: {self.ROE:.2%}

Based on this, what is your investment recommendation? Pick one action candidate."""
        self.prompt=prompt
        
    def generate_response(self):
        generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer
        )

        outputs = generator(
            self.prompt,
            max_new_tokens=500,         # Reduced for memory efficiency
            do_sample=True,
            temperature=0.4,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            # Memory efficient generation settings
            num_beams=1,                # No beam search to save memory
            #early_stopping=True,
            use_cache=True
        )

        generated_text = outputs[0]['generated_text']
        return generated_text


In [ ]:
class technical_analyst:

    def __init__(self, ticker):
        self.ticker=ticker
    
    def generate_df(self):
        self.data=yf.download(self.ticker, period='1y')

    def compute_rsi(close, period=14):
        delta = close.diff()

        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)

        avg_gain = gain.rolling(window=period).mean()
        avg_loss = loss.rolling(window=period).mean()

        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def compute_obv(close, volume):
        direction = close.diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
        obv = (volume * direction).fillna(0).cumsum()
        return obv
    
    def generate_indicators(self):
        temp=pd.DataFrame()
        temp["SMA_5"] = self.data["Close"].rolling(5).mean()
        temp["SMA_15"] = self.data["Close"].rolling(15).mean()
        temp["SMA_50"] = self.data["Close"].rolling(50).mean()

        temp['EMA_5'] = self.data['Close'].ewm(span=5).mean()
        temp['EMA_10'] = self.data['Close'].ewm(span=10).mean()
        temp['EMA_50'] = self.data['Close'].ewm(span=50).mean()
        temp["Date"] = self.data['Close'].index
        temp["RSI"]=self.compute_rsi(self.data['Close'])
        temp["OBV"]=self.compute_obv(self.data['Close'], self.data['Volume'])
        self.indicator_df=temp

    def create_technical_prompt(self):
        latest = self.indicator_df.iloc[-1]
        prompt = f"""
        You are a technical investment analyst. Analyze the recent technical performance of {self.ticker} and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
        Justify your decision in 4–6 bullet points using technical analysis. Only use the numerical data given. 
        Do not add assumptions about company fundamentals, operations, or strategy.

        Technical Indicators for {self.ticker} (most recent data point):

        - SMA 5: {latest['SMA_5']:.2f}
        - SMA 15: {latest['SMA_15']:.2f}
        - SMA 50: {latest['SMA_50']:.2f}

        - EMA 5: {latest['EMA_5']:.2f}
        - EMA 10: {latest['EMA_10']:.2f}
        - EMA 50: {latest['EMA_50']:.2f}

        - RSI: {latest['RSI']:.2f}
        - OBV: {latest['OBV']:,.0f}

        Based on this, what is your investment recommendation? Pick one action candidate.
        """
        self.prompt=prompt

    def generate_response(self):
        generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer
        )

        outputs = generator(
            self.prompt,
            max_new_tokens=500,         # Reduced for memory efficiency
            do_sample=True,
            temperature=0.4,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            # Memory efficient generation settings
            num_beams=1,                # No beam search to save memory
            #early_stopping=True,
            use_cache=True
        )

        generated_text = outputs[0]['generated_text']
        return generated_text


In [ ]:
class news_analyst:

    def __init__ (self,ticker):
        self.ticker=ticker
    
    def get_news_articles(self):
        to_date = datetime.today().date()

        from_date = to_date - timedelta(days=7)

        from_str = from_date.strftime('%Y-%m-%d')
        to_str = to_date.strftime('%Y-%m-%d')

        symbol = "AAPL"
        api_key = "d1l719pr01qt4thec1pgd1l719pr01qt4thec1q0"
        url = f"https://finnhub.io/api/v1/company-news?symbol={symbol}&from={from_str}&to={to_str}&token={api_key}"
        response = requests.get(url)
        news = response.json()
        filtered_news = [
            item for item in news
            if "apple" in item['headline'].lower()
        ]
        self.selected_news = random.sample(filtered_news, k=min(10, len(filtered_news)))
    
    def generate_news_prompt(self):
        prompt = f"""
You are a sentiment and headlines investment analyst. Analyze the recent technical performance of {t} and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
Justify your decision in 4–6 bullet points using sentiment analysis. Only use the headlines given. 
Do not add assumptions about company fundamentals, operations, or strategy.
Headlines:
"""
        for i, item in enumerate(self.selected_news):
            prompt+=item['headline']
            prompt+='\n'
        prompt+="Based on this, what is your investment recommendation? Pick one action candidate."
        self.prompt=prompt

    def generate_response(self):
        generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer
        )

        outputs = generator(
            self.prompt,
            max_new_tokens=500,         # Reduced for memory efficiency
            do_sample=True,
            temperature=0.4,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            # Memory efficient generation settings
            num_beams=1,                # No beam search to save memory
            #early_stopping=True,
            use_cache=True
        )

        generated_text = outputs[0]['generated_text']
        return generated_text